In [3]:
from glob import glob
from tqdm import tqdm_notebook as tqdm

import numpy as np
import pandas as pd

import torch

from matplotlib import pyplot as plt
import seaborn as sns

from transformers import BertTokenizer, BertModel

pd.set_option("display.max_rows", 300)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

In [19]:
e150_df = pd.read_pickle('../inputs/datasets/char_preds/e150/e150_char_preds.pkl')
e152_df = pd.read_pickle('../inputs/datasets/char_preds/e152/e152_char_preds.pkl')
e153_df = pd.read_pickle('../inputs/datasets/char_preds/e153/e153_char_preds.pkl')
e154_df = pd.read_pickle('../inputs/datasets/char_preds/e154/e154_char_preds.pkl')
e155_df = pd.read_pickle('../inputs/datasets/char_preds/e155/e155_char_preds.pkl')

In [15]:
trn_df = pd.read_csv('../inputs/origin/train.csv')
trn_df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [25]:
!mkdir ../inputs/datasets/guchio_char_preds/
!mkdir ../inputs/datasets/guchio_char_preds/e150
!mkdir ../inputs/datasets/guchio_char_preds/e152
!mkdir ../inputs/datasets/guchio_char_preds/e153
!mkdir ../inputs/datasets/guchio_char_preds/e154
!mkdir ../inputs/datasets/guchio_char_preds/e155

mkdir: cannot create directory '../inputs/datasets/guchio_char_preds/': File exists


In [26]:
for exp_id, df in zip(['e150', 'e152', 'e153', 'e154', 'e155'], [e150_df, e152_df, e153_df, e154_df, e155_df]):
    tmp_df = trn_df.merge(df, on='textID', how='left')
    tmp_df.loc[314, 'char_pred_start'] = np.zeros(141)
    tmp_df.loc[314, 'char_pred_end'] = np.zeros(141)
    np.save(f'../inputs/datasets/guchio_char_preds/{exp_id}/oof_char_start.npy', np.asarray(tmp_df.char_pred_start.tolist()), allow_pickle=False)
    np.save(f'../inputs/datasets/guchio_char_preds/{exp_id}/oof_char_end.npy', np.asarray(tmp_df.char_pred_end.tolist()), allow_pickle=False)

In [27]:
e154_df.head()

,textID,char_pred_start,char_pred_end
0,cb774db0d1,"[0.9982798099517822, 0.9982798099517822, 0.000...","[0.00024153896083589643, 0.0002415389608358964..."
1,549e992a42,"[0.3679257035255432, 0.3679257035255432, 0.367...","[0.0004281910660210997, 0.0004281910660210997,..."
2,088c60f138,"[0.3835388720035553, 0.3835388720035553, 0.383...","[0.0008378780912607908, 0.0008378780912607908,..."
3,9642c003ef,"[0.12908340990543365, 0.12908340990543365, 0.1...","[0.0006538929301314056, 0.0006538929301314056,..."
4,358bd9e861,"[0.544698178768158, 0.544698178768158, 0.54469...","[0.0010821548057720065, 0.0010821548057720065,..."


In [28]:
e155_df.head()

,textID,char_pred_start,char_pred_end
0,cb774db0d1,"[0.9982798099517822, 0.9982798099517822, 0.000...","[0.00024153896083589643, 0.0002415389608358964..."
1,549e992a42,"[0.3679257035255432, 0.3679257035255432, 0.367...","[0.0004281910660210997, 0.0004281910660210997,..."
2,088c60f138,"[0.3835388720035553, 0.3835388720035553, 0.383...","[0.0008378780912607908, 0.0008378780912607908,..."
3,9642c003ef,"[0.12908340990543365, 0.12908340990543365, 0.1...","[0.0006538929301314056, 0.0006538929301314056,..."
4,358bd9e861,"[0.544698178768158, 0.544698178768158, 0.54469...","[0.0010821548057720065, 0.0010821548057720065,..."
